# 使用全链接神经网络进行训练

## 1 导入数据

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error

In [2]:
train = pd.read_csv("../../kaggle/LosAngelesHousePricesForecast/data/train_1.csv")
if 'Unnamed: 0' in train.columns: 
    train = train.drop(['Unnamed: 0'], axis=1)
y = train["SalePrice"]
train1 = train.drop(["Id", "SalePrice"], axis=1)
X = pd.get_dummies(train1).reset_index(drop=True)
X_train, X_test, y_train, y_test = tts(X,y,test_size=0.2,random_state = 123)   

In [3]:
X.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0


### 评估函数

In [4]:
def benchmark(model):
    pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, pred))
    logrmse = np.sqrt(mean_squared_error(np.log(y_test), np.log(pred)))
    print("RMSE: {} \nLOGRMSE: {}".format(rmse, logrmse))
    return rmse, logrmse

## 2 模型训练

In [5]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import metrics
from keras import backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


In [6]:
keras.__version__

'2.2.4'

In [7]:
tf.__version__

'1.12.0'

In [8]:
def create_model():
    model = Sequential()
    model.add(Dense(10, input_dim=X_train.shape[1], activation="relu"))
    model.add(Dense(200,activation="relu")) 
    model.add(Dropout(0.1))
    model.add(Dense(50, activation="relu"))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [16]:
model = KerasRegressor(build_fn=create_model)

In [21]:
batch_size = [16, 32, 48, 64, 80, 96]
epochs = [100, 500, 1000, 2000, 4000]
param_grid = dict(batch_size=batch_size,epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=0, n_jobs=-1)
grid_res = grid.fit(X_train, y_train)
for mean,stdev,params in zip(grid_res.cv_results_['mean_test_score'],
                           grid_res.cv_results_['std_test_score'],
                           grid_res.cv_results_['params']):
    print("mean:{} stdev:{} params:{}".format(mean,stdev,params))
print("Best score: {} Best params:{}".format(grid_res.best_score_, grid_res.best_params_))

/Users/lyy/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Epoch 1/4000
1168/1168 [==============================] - 1s 1ms/step - loss: 37128520044.7123
Epoch 2/4000
1168/1168 [==============================] - 0s 90us/step - loss: 33108080836.3836
Epoch 3/4000
1168/1168 [==============================] - 0s 86us/step - loss: 20012834451.2877
Epoch 4/4000
1168/1168 [==============================] - 0s 91us/step - loss: 10677352798.6849
Epoch 5/4000
1168/1168 [==============================] - 0s 109us/step - loss: 9125996316.0548
Epoch 6/4000
1168/1168 [==============================] - 0s 81us/step - loss: 8820281035.3973
Epoch 7/4000
1168/1168 [==============================] - 0s 72us/step - loss: 7345560740.8219
Epoch 8/4000
1168/1168 [==============================] - 0s 73us/step - loss: 6881826896.6575
Epoch 9/4000
1168/1168 [==============================] - 0s 72us/step - loss: 6311752090.3014
Epoch 10/4000
1168/1168 [==============================] - 0s 72us/step - loss: 5849537257.2055
Epoch 11/4000
1168/1168 [====================

1168/1168 [==============================] - 0s 78us/step - loss: 1860937332.6027
Epoch 171/4000
1168/1168 [==============================] - 0s 81us/step - loss: 1842062542.0274
Epoch 172/4000
1168/1168 [==============================] - 0s 81us/step - loss: 1860566805.0411
Epoch 173/4000
1168/1168 [==============================] - 0s 84us/step - loss: 1861058543.7808
Epoch 174/4000
1168/1168 [==============================] - 0s 81us/step - loss: 1781273645.5890
Epoch 175/4000
1168/1168 [==============================] - 0s 79us/step - loss: 1791241226.5205
Epoch 176/4000
1168/1168 [==============================] - 0s 78us/step - loss: 1819552074.5205
Epoch 177/4000
1168/1168 [==============================] - 0s 80us/step - loss: 1874410235.6164
Epoch 178/4000
1168/1168 [==============================] - 0s 85us/step - loss: 1811389158.5753
Epoch 179/4000
1168/1168 [==============================] - 0s 82us/step - loss: 1838018362.7397
Epoch 180/4000
1168/1168 [===================

1168/1168 [==============================] - 0s 78us/step - loss: 1418177200.2192
Epoch 339/4000
1168/1168 [==============================] - 0s 78us/step - loss: 1410951530.9589
Epoch 340/4000
1168/1168 [==============================] - 0s 78us/step - loss: 1388163540.1644
Epoch 341/4000
1168/1168 [==============================] - 0s 78us/step - loss: 1406481787.6164
Epoch 342/4000
1168/1168 [==============================] - 0s 78us/step - loss: 1404847317.0411
Epoch 343/4000
1168/1168 [==============================] - 0s 78us/step - loss: 1355165425.0959
Epoch 344/4000
1168/1168 [==============================] - 0s 77us/step - loss: 1498400159.1233
Epoch 345/4000
1168/1168 [==============================] - 0s 77us/step - loss: 1425775941.2603
Epoch 346/4000
1168/1168 [==============================] - 0s 77us/step - loss: 1416687210.0822
Epoch 347/4000
1168/1168 [==============================] - 0s 77us/step - loss: 1481056047.3425
Epoch 348/4000
1168/1168 [===================

1168/1168 [==============================] - 0s 75us/step - loss: 1110538583.2329
Epoch 507/4000
1168/1168 [==============================] - 0s 75us/step - loss: 1121913654.7945
Epoch 508/4000
1168/1168 [==============================] - 0s 75us/step - loss: 1140083778.6301
Epoch 509/4000
1168/1168 [==============================] - 0s 77us/step - loss: 1166484665.4247
Epoch 510/4000
1168/1168 [==============================] - 0s 75us/step - loss: 1133701967.7808
Epoch 511/4000
1168/1168 [==============================] - 0s 76us/step - loss: 1125867171.5068
Epoch 512/4000
1168/1168 [==============================] - 0s 76us/step - loss: 1154336389.6986
Epoch 513/4000
1168/1168 [==============================] - 0s 75us/step - loss: 1134075165.8082
Epoch 514/4000
1168/1168 [==============================] - 0s 75us/step - loss: 1137398659.9452
Epoch 515/4000
1168/1168 [==============================] - 0s 75us/step - loss: 1143893394.8493
Epoch 516/4000
1168/1168 [===================

...

1168/1168 [==============================] - 0s 100us/step - loss: 296513656.1096
Epoch 3200/4000
1168/1168 [==============================] - 0s 100us/step - loss: 277079050.9589
Epoch 3201/4000
1168/1168 [==============================] - 0s 93us/step - loss: 293107219.5068
Epoch 3202/4000
1168/1168 [==============================] - 0s 90us/step - loss: 280107768.7671
Epoch 3203/4000
1168/1168 [==============================] - 0s 90us/step - loss: 281913479.6712
Epoch 3204/4000
1168/1168 [==============================] - 0s 86us/step - loss: 283131598.2466
Epoch 3205/4000
1168/1168 [==============================] - 0s 86us/step - loss: 266352318.0274
Epoch 3206/4000
1168/1168 [==============================] - 0s 81us/step - loss: 310531317.6986
Epoch 3207/4000
1168/1168 [==============================] - 0s 83us/step - loss: 273769596.3836
Epoch 3208/4000
1168/1168 [==============================] - 0s 88us/step - loss: 306473960.9863
Epoch 3209/4000
1168/1168 [=================

1168/1168 [==============================] - 0s 90us/step - loss: 275146046.5753
Epoch 3368/4000
1168/1168 [==============================] - 0s 90us/step - loss: 286323146.7397
Epoch 3369/4000
1168/1168 [==============================] - 0s 91us/step - loss: 303456081.0959
Epoch 3370/4000
1168/1168 [==============================] - 0s 90us/step - loss: 286449346.4110
Epoch 3371/4000
1168/1168 [==============================] - 0s 89us/step - loss: 310107294.5753
Epoch 3372/4000
1168/1168 [==============================] - 0s 87us/step - loss: 271506418.6301
Epoch 3373/4000
1168/1168 [==============================] - 0s 85us/step - loss: 273035870.6849
Epoch 3374/4000
1168/1168 [==============================] - 0s 90us/step - loss: 291917671.4521
Epoch 3375/4000
1168/1168 [==============================] - 0s 89us/step - loss: 337602516.3836
Epoch 3376/4000
1168/1168 [==============================] - 0s 90us/step - loss: 292394184.6575
Epoch 3377/4000
1168/1168 [===================

1168/1168 [==============================] - 0s 85us/step - loss: 286469027.9452
Epoch 3536/4000
1168/1168 [==============================] - 0s 83us/step - loss: 296651394.7397
Epoch 3537/4000
1168/1168 [==============================] - 0s 86us/step - loss: 299076034.6301
Epoch 3538/4000
1168/1168 [==============================] - 0s 90us/step - loss: 287076659.9452
Epoch 3539/4000
1168/1168 [==============================] - 0s 88us/step - loss: 289083369.6438
Epoch 3540/4000
1168/1168 [==============================] - 0s 87us/step - loss: 279716587.3973
Epoch 3541/4000
1168/1168 [==============================] - 0s 88us/step - loss: 273189188.9315
Epoch 3542/4000
1168/1168 [==============================] - 0s 84us/step - loss: 293690828.9315
Epoch 3543/4000
1168/1168 [==============================] - 0s 84us/step - loss: 280473036.4932
Epoch 3544/4000
1168/1168 [==============================] - 0s 83us/step - loss: 280930390.4658
Epoch 3545/4000
1168/1168 [===================

1168/1168 [==============================] - 0s 84us/step - loss: 253351981.6986
Epoch 3704/4000
1168/1168 [==============================] - 0s 82us/step - loss: 295093789.5890
Epoch 3705/4000
1168/1168 [==============================] - 0s 84us/step - loss: 301631746.5205
Epoch 3706/4000
1168/1168 [==============================] - 0s 89us/step - loss: 271009251.7260
Epoch 3707/4000
1168/1168 [==============================] - 0s 87us/step - loss: 266669701.8082
Epoch 3708/4000
1168/1168 [==============================] - 0s 88us/step - loss: 294163762.6301
Epoch 3709/4000
1168/1168 [==============================] - 0s 87us/step - loss: 266954021.4795
Epoch 3710/4000
1168/1168 [==============================] - 0s 83us/step - loss: 271819693.5890
Epoch 3711/4000
1168/1168 [==============================] - 0s 85us/step - loss: 271430149.2603
Epoch 3712/4000
1168/1168 [==============================] - 0s 87us/step - loss: 321456706.4110
Epoch 3713/4000
1168/1168 [===================

1168/1168 [==============================] - 0s 82us/step - loss: 340387474.1918
Epoch 3872/4000
1168/1168 [==============================] - 0s 84us/step - loss: 346490040.5479
Epoch 3873/4000
1168/1168 [==============================] - 0s 83us/step - loss: 362603044.1644
Epoch 3874/4000
1168/1168 [==============================] - 0s 80us/step - loss: 388333111.6712
Epoch 3875/4000
1168/1168 [==============================] - 0s 87us/step - loss: 344543326.4658
Epoch 3876/4000
1168/1168 [==============================] - 0s 89us/step - loss: 348192923.8356
Epoch 3877/4000
1168/1168 [==============================] - 0s 86us/step - loss: 337775382.0274
Epoch 3878/4000
1168/1168 [==============================] - 0s 85us/step - loss: 332723412.6027
Epoch 3879/4000
1168/1168 [==============================] - 0s 87us/step - loss: 320225293.5890
Epoch 3880/4000
1168/1168 [==============================] - 0s 86us/step - loss: 335539209.7534
Epoch 3881/4000
1168/1168 [===================

In [22]:
# Best score: -951423830.0616438 Best params:{'batch_size': 48, 'epochs': 4000}

In [30]:
history = model.fit(X_train, y_train, epochs=500, batch_size=32, verbose=0)

KeyboardInterrupt: 

In [24]:
benchmark(model)

RMSE: 30255.047113464894 
LOGRMSE: 0.13724170239696631


(30255.047113464894, 0.13724170239696631)

In [9]:
def create_model():
    model = Sequential()
    model.add(Dense(10, input_dim=X_train.shape[1], activation="relu"))
    model.add(Dense(300, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [10]:
model = create_model()

In [11]:
model.fit(X_train, y_train, epochs=3500, batch_size=32, verbose=0)
# 3500,32 0.128
# 4000,32,0.131
# 3000,32,0.131
# 3200,32,0.241


In [12]:
benchmark(model)

RMSE: 29594.705875833213 
LOGRMSE: 0.13145692675453338


(29594.705875833213, 0.13145692675453338)

In [14]:
# import matplotlib.pyplot as plt 
# %matplotlib inline
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.show()

In [25]:
def create_model():
    model = Sequential()
    model.add(Dense(10, input_dim=X_train.shape[1], activation="relu"))
    model.add(Dense(300, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [26]:
model = create_model()

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 10)                3040      
_________________________________________________________________
dense_30 (Dense)             (None, 300)               3300      
_________________________________________________________________
dropout_8 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 50)                15050     
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 51        
Total params: 21,441
Trainable params: 21,441
Non-trainable params: 0
_________________________________________________________________


In [28]:
history = model.fit(X_train, y_train, epochs=2000, batch_size=32, verbose=0)

In [29]:
benchmark(model)

RMSE: 30301.045829143353 
LOGRMSE: 0.14016619832623473


(30301.045829143353, 0.14016619832623473)

In [20]:
model=create_model()
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=3350,batch_size=32,verbose=0)

In [21]:
benchmark(model)

RMSE: 31665.02579389985 
LOGRMSE: 0.14228086659366299


(31665.02579389985, 0.14228086659366299)